In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

symbol = "ASML.AS"
start_date = "2018-01-01"
end_date = "2023-12-31"
df = yf.download(symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
def hurst_fd(price_series, min_window=10, max_window=100, num_windows=20, num_samples=100):
    log_returns = np.diff(np.log(price_series))
    window_sizes = np.linspace(min_window, max_window, num_windows, dtype=int)
    R_S = []

    for w in window_sizes:
        R, S = [], []
        for _ in range(num_samples):
            start = np.random.randint(0, len(log_returns) - w)
            seq = log_returns[start:start + w]
            R.append(np.max(seq) - np.min(seq))
            S.append(np.std(seq))

        R_S.append(np.mean(R) / np.mean(S))

    log_window_sizes = np.log(window_sizes)
    log_R_S = np.log(R_S)
    coeffs = np.polyfit(log_window_sizes, log_R_S, 1)
    hurst_exponent = coeffs[0]
    fractal_dimension = 2 - hurst_exponent

    return hurst_exponent, fractal_dimension

def rolling_hurst(price_series, window, min_window=10, max_window=100, num_windows=20, num_samples=100):
    return price_series.rolling(window=window).apply(lambda x: hurst_fd(x, min_window, max_window, num_windows, num_samples)[0], raw=True)

def rolling_fractal_dimension(price_series, window, min_window=10, max_window=100, num_windows=20, num_samples=100):
    return price_series.rolling(window=window).apply(lambda x: hurst_fd(x, min_window, max_window, num_windows, num_samples)[1], raw=True)

In [1]:
# Essential Libraries Import
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Function to calculate Hurst Exponent and Fractal Dimension
def hurst_fd(price_series, min_window=10, max_window=100, num_windows=20, num_samples=100):
    # Calculate logarithmic returns
    log_returns = np.diff(np.log(price_series))

    # Define window sizes for the resampling approach
    window_sizes = np.linspace(min_window, max_window, num_windows, dtype=int)
    R_S = []

    # For each window size, calculate R/S values
    for w in window_sizes:
        R, S = [], []

        # Resample multiple sequences for a more robust estimate
        for _ in range(num_samples):
            start = np.random.randint(0, len(log_returns) - w)
            seq = log_returns[start:start + w]
            R.append(np.max(seq) - np.min(seq))
            S.append(np.std(seq))

        R_S.append(np.mean(R) / np.mean(S))

    # Calculate the slope (Hurst Exponent) in a log-log plot
    log_window_sizes = np.log(window_sizes)
    log_R_S = np.log(R_S)
    coeffs = np.polyfit(log_window_sizes, log_R_S, 1)

    # Extract Hurst Exponent and derive Fractal Dimension
    hurst_exponent = coeffs[0]
    fractal_dimension = 2 - hurst_exponent

    return hurst_exponent, fractal_dimension

# Function to calculate rolling Hurst Exponent
def rolling_hurst(price_series, window, min_window=10, max_window=100, num_windows=20, num_samples=100):
    return price_series.rolling(window=window).apply(lambda x: hurst_fd(x, min_window, max_window, num_windows, num_samples)[0], raw=True)

# Function to calculate rolling Fractal Dimension
def rolling_fractal_dimension(price_series, window, min_window=10, max_window=100, num_windows=20, num_samples=100):
    return price_series.rolling(window=window).apply(lambda x: hurst_fd(x, min_window, max_window, num_windows, num_samples)[1], raw=True)

# Define symbol and fetch its historical data
symbol = "ASML.AS"
start_date = "2018-01-01"
end_date = "2026-12-27"
df = yf.download(symbol, start=start_date, end=end_date)

# Calculate rolling Hurst Exponent and Fractal Dimension for the stock data
rolling_window = 120
df['hurst_exponent'] = rolling_hurst(df['Close'], rolling_window)
df['fractal_dimension'] = rolling_fractal_dimension(df['Close'], rolling_window)
df.dropna(inplace=True)

# Calculate 30-day Simple Moving Average for visualization purposes
sma_window = 30
df['sma'] = df['Close'].rolling(window=sma_window).mean()

# Calculate mean price for visualization purposes
mean_price = df['Close'].mean()

[*********************100%%**********************]  1 of 1 completed


In [3]:
plt.show()